In [ ]:
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from google.colab import drive
import os

In [ ]:
vidcap = cv2.VideoCapture('/content/movietest.mp4')
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite('/content/gdrive/MyDrive/Final_Project_Super Resolution/VideoImage/'+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 0.1 #//it will capture image in each 0.5 second
count=1
success = getFrame(sec)
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

In [ ]:
#Mount drive to Google Colab

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model = keras.models.load_model('/content/gdrive/MyDrive/Final_Project_Super Resolution/Results/superresolution3.h5')

In [ ]:
import cv2
import numpy as np
import os
from os.path import isfile, join
pathIn= '/content/gdrive/MyDrive/Final_Project_Super Resolution/VideoImage/'
pathOut = 'video24.mp4'
fps = 9.9
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
files.sort()
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
for i in range(len(files)):
    filename=pathIn + files[i]
    #reading each files
    #img=cv2.imread(filename)
    img=process_images(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

In [ ]:
def process_images(filepath):

    target = cv2.imread(filepath, cv2.IMREAD_COLOR)
    target = cv2.cvtColor(target, cv2.COLOR_BGR2YCrCb)
    shape = target.shape

    Y_img = cv2.resize(target[:, :, 0], (int(shape[1] / 1), int(shape[0] / 1)), cv2.INTER_CUBIC)

    # Resize up to orignal image
    Y_img = cv2.resize(Y_img, (shape[1], shape[0]), cv2.INTER_CUBIC)
    target[:, :, 0] = Y_img
    target = cv2.cvtColor(target, cv2.COLOR_YCrCb2BGR)

    Y = np.zeros((1, target.shape[0], target.shape[1], 1), dtype=np.float32)

    # Normalize
    Y[0, :, :, 0] = Y_img.astype(np.float32) / 255.

    # Predict
    pre = model.predict(Y, batch_size=1) * 255.

    # Post process output
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)

    # Copy y channel back to image and convert to BGR
    output = cv2.cvtColor(target, cv2.COLOR_BGR2YCrCb)
    output[6: -6, 6: -6, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(output, cv2.COLOR_YCrCb2BGR)
    return output


In [ ]:
for filename in os.listdir('/content/gdrive/MyDrive/Final_Project_Super Resolution/VideoImage/'):
    folder='/content/gdrive/MyDrive/Final_Project_Super Resolution/VideoImage'
    path=os.path.join(folder,filename)
    output=process_images(path)
    cv2.imwrite('/content/gdrive/MyDrive/Final_Project_Super Resolution/upscaledvideo/'+filename,output)
  